# Preparando os dados do IPTU recorrente

Esse processamento é realizado recorrente a cada ano, LER NO README

In [1]:
import vaex
import geopandas as gpd
from shapely.validation import make_valid
from difflib import SequenceMatcher
import numpy as np
import pandas as pd
# import pygeos

/home/fernando/dev/dash-iptu/venv/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
gdf_distritos = gpd.read_file('data/SIRGAS_GPKG_distrito.gpkg')

gdf_quadras = gpd.read_file('zip://data/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp')
gdf_quadras = gdf_quadras.set_crs(epsg=31983)
gdf_quadras = gdf_quadras[gdf_quadras.qd_tipo == 'F']
gdf_quadras = gdf_quadras.dissolve(['qd_setor', 'qd_fiscal']).reset_index()
gdf_quadras_centroid = gdf_quadras.copy()
gdf_quadras_centroid.geometry = gdf_quadras_centroid.representative_point()

In [3]:
gdf_quadras.shape, gdf_quadras_centroid.shape

((45655, 10), (45655, 10))

In [4]:
gdf_quadras_distritos = gdf_quadras_centroid.sjoin(gdf_distritos, how='left', predicate='intersects')
gdf_quadras_distritos.drop('index_right', axis=1, inplace=True)

In [5]:
gdf_quadras.shape, gdf_quadras_distritos.shape

((45655, 10), (45655, 15))

In [6]:
gdf_quadras_distritos['sq'] = gdf_quadras_distritos.qd_setor + gdf_quadras_distritos.qd_fiscal

In [7]:
gpd.GeoDataFrame(gdf_quadras_distritos[['sq', 'ds_codigo', 'ds_nome']], geometry=gdf_quadras.geometry,
                crs='EPSG:31983').to_file('data/quadras.gpkg')

/home/fernando/dev/dash-iptu/venv/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [8]:
gdf_od = gpd.read_file('zip://data/SIRGAS_SHP_origemdestino_2017.zip!SIRGAS_SHP_origemdestino_2017')
gdf_od_sp = gdf_od[gdf_od.od_municip == '36']
gdf_od_sp.set_crs(epsg=31983, inplace=True)
gdf_quadras_distritos_od = gdf_quadras_distritos.sjoin(gdf_od_sp, how='left', predicate='intersects')

df_quadras_distritos_od = gdf_quadras_distritos_od.loc[:, ['qd_setor', 'qd_fiscal', 'ds_codigo', 'ds_nome', 'ds_cd_sub', 'ds_subpref', 'od_id', 'od_nome']]
df_quadras_distritos_od.loc[:, ['sq']] = df_quadras_distritos_od.qd_setor + df_quadras_distritos_od.qd_fiscal
# df_quadras_distritos_od.drop_duplicates(keep='first', inplace=True, ignore_index=True)

/home/fernando/dev/dash-iptu/venv/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [9]:
gdf_distritos.sort_values('ds_nome')

,ds_codigo,ds_subpref,ds_cd_sub,ds_sigla,ds_nome,geometry
38,1,MOOCA,25,ARA,AGUA RASA,"POLYGON ((341093.380 7391912.491, 341093.329 7..."
55,2,PINHEIROS,11,API,ALTO DE PINHEIROS,"POLYGON ((326739.457 7395585.840, 326741.473 7..."
70,3,PERUS,01,ANH,ANHANGUERA,"POLYGON ((318695.858 7403032.323, 318677.730 7..."
43,4,ARICANDUVA-FORMOSA-CARRAO,26,ARI,ARICANDUVA,"POLYGON ((344068.781 7390960.855, 344068.149 7..."
11,5,PENHA,21,AAL,ARTUR ALVIM,"POLYGON ((349416.226 7397718.808, 349416.884 7..."
...,...,...,...,...,...,...
77,90,VILA MARIANA,12,VMN,VILA MARIANA,"POLYGON ((332359.740 7389124.109, 332309.630 7..."
14,91,PENHA,21,VMT,VILA MATILDE,"POLYGON ((341949.933 7396395.900, 341934.334 7..."
10,92,VILA MARIA-VILA GUILHERME,07,VMD,VILA MEDEIROS,"POLYGON ((340579.417 7401672.553, 340579.493 7..."
47,93,VILA PRUDENTE,29,VPR,VILA PRUDENTE,"POLYGON ((341731.295 7388459.495, 341723.755 7..."


In [10]:
gdf_quadras.shape, df_quadras_distritos_od.shape

((45655, 10), (45655, 9))

In [11]:
gdf_ponderacao_censo = gpd.read_file('data/areas-ponderacao-censo.gpkg', layer='areas-ponderacao-censo-2010')
df_quadras_censo =  gdf_quadras_centroid.sjoin(gdf_ponderacao_censo, how='left', predicate='intersects')

In [12]:
df_quadras_distritos_od.shape, df_quadras_censo[['COD_AED', 'COD_AED_S']].shape

((45655, 9), (45655, 2))

In [13]:
df_quadras_distritos_od = pd.concat(
    [df_quadras_distritos_od,
    df_quadras_censo[['COD_AED', 'COD_AED_S']]],
    axis=1
)

In [14]:
df_quadras_distritos_od.columns

Index(['qd_setor', 'qd_fiscal', 'ds_codigo', 'ds_nome', 'ds_cd_sub',
       'ds_subpref', 'od_id', 'od_nome', 'sq', 'COD_AED', 'COD_AED_S'],
      dtype='object')

In [15]:
gdf_od_sp = gdf_od_sp.to_file('data/OD-sp.gpkg', driver='GPKG')

/home/fernando/dev/dash-iptu/venv/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [16]:
df_quadras_distritos_od

,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,sq,COD_AED,COD_AED_S
0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003,3550308005004,4
1,001,004,9,BOM RETIRO,09,SE,7,Luz,001004,3550308005004,4
2,001,005,9,BOM RETIRO,09,SE,7,Luz,001005,3550308005004,4
3,001,006,9,BOM RETIRO,09,SE,7,Luz,001006,3550308005004,4
4,001,007,9,BOM RETIRO,09,SE,7,Luz,001007,3550308005004,4
...,...,...,...,...,...,...,...,...,...,...,...
45650,310,116,38,JABAQUARA,15,JABAQUARA,274,Jabaquara,310116,3550308005095,95
45651,310,117,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310117,3550308005093,93
45652,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118,3550308005093,93
45653,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310119,3550308005093,93


In [17]:
## TODO
# Verificar essa inconsistencia
gdf_quadras_distritos[gdf_quadras_distritos.ds_codigo.isna()]

,qd_setor,qd_fiscal,geometry,qd_id_orig,qd_tipo,qd_subqua,qd_area,qd_situac,qd_id,qd_tx_tipo,ds_codigo,ds_subpref,ds_cd_sub,ds_sigla,ds_nome,sq
27467,133,127,POINT (357031.736 7402052.859),10122413,F,001,65776.7480405944,1,179963,FISCAL,NaN,NaN,NaN,NaN,NaN,133127
33243,156,135,POINT (343167.543 7387776.019),8985852,F,001,2484.92461758186,1,148338,FISCAL,NaN,NaN,NaN,NaN,NaN,156135
37127,173,160,POINT (333525.605 7378933.173),9704470,F,001,1348.82739798853,1,182464,FISCAL,NaN,NaN,NaN,NaN,NaN,173160
37132,173,165,POINT (333590.700 7378465.176),9704234,F,001,2641.57734002054,1,186397,FISCAL,NaN,NaN,NaN,NaN,NaN,173165


In [18]:
exercicio_inicio = 1995
exercicio_fim = 2023

In [19]:
dfs = []

for i in range(exercicio_inicio,exercicio_fim + 1):

    print(f'Processando {i}')
    
    # df = vaex.open(f'data/IPTU_{i}/IPTU_{i}.hdf5')
    df = vaex.open(f'data//IPTU-HDF5/IPTU_{i}/IPTU_{i}.hdf5')

    df.setor = df['NUMERO DO CONTRIBUINTE'].str.slice(0,3)
    df.quadra = df['NUMERO DO CONTRIBUINTE'].str.slice(3,6)
    df.fillna(value='00-0', column_names=['NUMERO DO CONDOMINIO'], inplace=True)
    df.fillna(value=1., column_names=['FRACAO IDEAL'], inplace=True)
    df.fillna(value=0., column_names=['AREA CONSTRUIDA', 'AREA OCUPADA', 'AREA DO TERRENO'], inplace=True)
    df.sqlc = df.func.where(df['NUMERO DO CONDOMINIO'] == '00-0',
                                    df['NUMERO DO CONTRIBUINTE'].str.slice(0, 10) + '00',
                                    df['NUMERO DO CONTRIBUINTE'].str.slice(0, 6) + '0000' + df['NUMERO DO CONDOMINIO'].str.slice(0, 2))
    df['sq'] = df.setor + df.quadra
    df['sqlc'] = df.sqlc
    df_sqlc_ac = df.groupby('sqlc', agg={'area_contruida_total': vaex.agg.sum('AREA CONSTRUIDA'),
                                         'area_ocupada_proporcional': vaex.agg.max('AREA OCUPADA')})
    df = df.join(df_sqlc_ac, on='sqlc')
    df['fracao_ideal'] = df.func.where(df['FRACAO IDEAL'] == 0.,  
                                                df['AREA CONSTRUIDA'] / df['area_contruida_total'],
                                                df['FRACAO IDEAL'])
    df['area_ocupada_proporcional'] = df['area_ocupada_proporcional'] * df['fracao_ideal']
    df['area_terreno_proporcional'] = df['AREA DO TERRENO'] * df['fracao_ideal']
    df['ca'] = (df['AREA CONSTRUIDA'] / df['fracao_ideal']) / df['AREA DO TERRENO']
    df['to'] = df['AREA OCUPADA'] / df['AREA DO TERRENO']

    df['valor_terreno'] = df['fracao_ideal'] * df['VALOR DO M2 DO TERRENO'] * df['AREA DO TERRENO']
    df['valor_construcao'] = df['VALOR DO M2 DE CONSTRUCAO'] * df['AREA CONSTRUIDA']

    dfs.append(df)
    # break


Processando 1995


NameError: Column or variable 'area_terreno_proporcional' does not exist. Did you mean: 'area_ocupada_proporcional' or '__area_ocupada_proporcional'

In [ ]:
df[['AREA OCUPADA', 'area_ocupada_proporcional', 'area_contruida_total', 'AREA CONSTRUIDA']]

/home/fernando/dev/dash-iptu/venv/lib/python3.9/site-packages/vaex/arrow/numpy_dispatch.py:72: RuntimeWarning: invalid value encountered in true_divide
  result_data = op['op'](a_data, b_data)


#,AREA OCUPADA,area_ocupada,area_contruida_total,AREA CONSTRUIDA
0,108,108.0,135,135
1,67,67.0,67,67
2,84,84.0,140,140
3,86,86.0,103,103
4,110,110.0,170,170
...,...,...,...,...
"3,639,371",5312,17.5296,25215,84
"3,639,372",5312,17.5296,25215,84
"3,639,373",5312,17.5296,25215,84
"3,639,374",0,0.0,0,0


In [ ]:
df_iptu = vaex.concat(dfs)

In [ ]:
df_iptu.shape

(85961435, 38)

In [ ]:
columns_to_drop = [
    'NUMERO DO CONTRIBUINTE',
    'NUMERO DA NL',
    'DATA DO CADASTRAMENTO',
    'NOME DE LOGRADOURO DO IMOVEL', 
    'NUMERO DO IMOVEL', 
    'COMPLEMENTO DO IMOVEL',
    'BAIRRO DO IMOVEL', 
    'REFERENCIA DO IMOVEL',
    'CEP DO IMOVEL',
    'QUANTIDADE DE ESQUINAS FRENTES',
    'VALOR DO M2 DO TERRENO',
    'VALOR DO M2 DE CONSTRUCAO' , 
    'TIPO DE TERRENO', 
    'ANO DE INICIO DA VIDA DO CONTRIBUINTE',
    'MES DE INICIO DA VIDA DO CONTRIBUINTE', 
    'FASE DO CONTRIBUINTE', 
    'FRACAO IDEAL'
]

In [ ]:
df_iptu.drop(columns=columns_to_drop, inplace=True)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,area_ocupada,ca,to,valor_terreno,valor_construcao
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.0,0.9926470588235294,0.0,13600.0,17415.0
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.0,0.7444444444444445,0.0,9000.0,6298.0
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.0,0.8095238095238095,0.0,10500.0,7990.0
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.0,0.9537037037037037,0.0,10800.0,13287.0
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.0,0.8166666666666667,0.0,12000.0,9212.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"85,961,430",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0
"85,961,431",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0
"85,961,432",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0
"85,961,433",2023,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,0.0,41939500.0,0.0


In [ ]:
df_iptu.shape

(85961435, 21)

In [ ]:
## Parece que para alguns lotes condominiais o terreno está proporcional à fracao ideal
df_iptu[df_iptu.ca > 60]['ca', 'AREA CONSTRUIDA', 'fracao_ideal', 'AREA DO TERRENO', 'area_contruida_total', 'ANO DO EXERCICIO']

#,ca,AREA CONSTRUIDA,fracao_ideal,AREA DO TERRENO,area_contruida_total,ANO DO EXERCICIO
0,62.370370370370374,99,0.01175771971496437,135,8420,1995
1,62.370370370370374,95,0.011282660332541567,135,8420,1995
2,62.370370370370374,106,0.012589073634204275,135,8420,1995
3,62.370370370370374,95,0.011282660332541567,135,8420,1995
4,62.370370370370374,106,0.012589073634204275,135,8420,1995
...,...,...,...,...,...,...
"2,418",62.398263700488336,23,0.0001,3686,31879,2023
"2,419",62.398263700488336,23,0.0001,3686,31879,2023
"2,420",62.398263700488336,23,0.0001,3686,31879,2023
"2,421",62.398263700488336,23,0.0001,3686,31879,2023


In [ ]:
df_iptu[df_iptu['NUMERO DO CONDOMINIO'] == '00-0'].shape

(46131796, 21)

In [ ]:
df_iptu.shape

(85961435, 21)

In [ ]:
df_quadras_distritos_od['sqod'] = df_quadras_distritos_od.sq + df_quadras_distritos_od.od_id.str.pad(width=3, side='left', fillchar='0')

In [ ]:
df_quadras_distritos_od.shape

(45655, 12)

In [ ]:
df_quadras_distritos_od

,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,sq,COD_AED,COD_AED_S,sqod
0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003,3550308005004,4,001003007
1,001,004,9,BOM RETIRO,09,SE,7,Luz,001004,3550308005004,4,001004007
2,001,005,9,BOM RETIRO,09,SE,7,Luz,001005,3550308005004,4,001005007
3,001,006,9,BOM RETIRO,09,SE,7,Luz,001006,3550308005004,4,001006007
4,001,007,9,BOM RETIRO,09,SE,7,Luz,001007,3550308005004,4,001007007
...,...,...,...,...,...,...,...,...,...,...,...,...
45650,310,116,38,JABAQUARA,15,JABAQUARA,274,Jabaquara,310116,3550308005095,95,310116274
45651,310,117,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310117,3550308005093,93,310117275
45652,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118,3550308005093,93,310118275
45653,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310119,3550308005093,93,310119275


In [ ]:
df_quadras_distritos_od = df_quadras_distritos_od.drop_duplicates(subset=['sq'])

In [ ]:
df_quadras_distritos_od.shape, df_iptu.shape

((45655, 12), (85961435, 21))

In [ ]:
df_iptu = df_iptu.join(vaex.from_pandas(df_quadras_distritos_od), on='sq', how='left',  
            allow_duplication=False, inplace=False)

In [ ]:
df_quadras_distritos_od.shape, df_iptu.shape

((45655, 12), (85961435, 32))

In [ ]:
df_usos_2022 = df_iptu[df_iptu['ANO DO EXERCICIO'] == 2022].groupby('TIPO DE USO DO IMOVEL').agg('count').to_pandas_df()
df_usos_2022

,TIPO DE USO DO IMOVEL,count
0,Flat residencial em condomínio,5184
1,Templo,5974
2,Loja em edifício em condomínio (unidade autônoma),23752
3,Terreno,103197
4,Oficina,9921
5,"Hospital, ambulatório, casa de saúde e assemel...",1352
6,Loja,54341
7,"Prédio de escritório ou consultório, não em co...",24185
8,Residência,1085600
9,"Outras edificações de uso comercial, com utili...",22163


In [ ]:
df_usos = df_iptu.groupby('TIPO DE USO DO IMOVEL').agg('count').to_pandas_df()

In [ ]:
df_usos.fillna('', inplace=True)

In [ ]:
results, ratios = [], []

for i, row in df_usos.iterrows():

    similaridade = df_usos_2022['TIPO DE USO DO IMOVEL'].apply(lambda x: SequenceMatcher(None, row['TIPO DE USO DO IMOVEL'], x).ratio())
    
    results.append(df_usos_2022.iloc[similaridade.idxmax()]['TIPO DE USO DO IMOVEL'])

    ratios.append(similaridade.max())

df_usos['titulo_padronizado'] = results # Os títulos costumam variar, portanto esse campo será referência para os agrupamentos
df_usos['relacao_de_assertividade_com_titulo_padronizado'] = ratios

In [ ]:
df_usos

,TIPO DE USO DO IMOVEL,count,titulo_padronizado,relacao_de_assertividade_com_titulo_padronizado
0,,26,Flat residencial em condomínio,0.000000
1,clube esportivo,11697,Clube esportivo,0.933333
2,loja em edifício em condomínio,436348,Loja em edifício em condomínio (unidade autônoma),0.734177
3,"outras edificações do tipo (uso comércio), com...",366569,"Outras edificações de uso comercial, com utili...",0.873016
4,Flat residencial em condomínio,39631,Flat residencial em condomínio,1.000000
...,...,...,...,...
73,"Asilo, orfanato, creche, seminário ou convento",6795,"Asilo, orfanato, creche, seminário ou convento",1.000000
74,Garagem (unidade autônoma) de prédio de garagens,75282,Garagem (unidade autônoma) de prédio de garagens,1.000000
75,"Prédio de apartamento, não em condomínio, de u...",10215,"Prédio de apartamento, não em condomínio, de u...",0.986486
76,Escola,54724,Escola,1.000000


In [ ]:
residencial = ['residencial', 'residência', 'apartamento', 'cortiço']
comercial = ['comércio', 'comercial', 'posto', 'loja']
servicos =  ['hotel', 'radioemissora', 'flat', 'serviço', 'clube', 'oficina', 'hospital', 'templo', 'escritórios', 'consultórios', 'escritório', 'consultório', 'asilo', 'creche', 'prédio de garagens', 'escola']
industrial = ['Indústria', 'armazéns']

In [ ]:
class_residencial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(residencial)), case=False)
class_comercial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(comercial)), case=False)
class_servicos = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(servicos)), case=False)
class_industrial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(industrial)), case=False)

df_usos.loc[class_residencial, ['classificacao_abrangente']] = 'Residencial'
df_usos.loc[class_comercial, ['classificacao_abrangente']] = 'Comercial'
df_usos.loc[class_servicos, ['classificacao_abrangente']] = 'Serviços'
df_usos.loc[class_industrial, ['classificacao_abrangente']] = 'Industrial'

df_usos.loc[df_usos.classificacao_abrangente.isna(), ['classificacao_abrangente']] = 'Outros'

In [ ]:
df_iptu['TIPO DE USO DO IMOVEL'].isna().value_counts()

False    85961409
True           26
dtype: int64

In [ ]:
# df_iptu['TIPO DE USO DO IMOVEL']
df_usos = vaex.from_pandas(df_usos.loc[df_usos['TIPO DE USO DO IMOVEL'] != '', ['TIPO DE USO DO IMOVEL', 'classificacao_abrangente']])
df_iptu = df_iptu.join(df_usos, on='TIPO DE USO DO IMOVEL')

In [ ]:
df_iptu[['classificacao_abrangente']]

#,classificacao_abrangente
0,Comercial
1,Comercial
2,Comercial
3,Comercial
4,Comercial
...,...
"85,961,430",Residencial
"85,961,431",Residencial
"85,961,432",Residencial
"85,961,433",Outros


In [ ]:
df_iptu.shape

(85961435, 33)

In [ ]:
df_padroes_usos_2022 = df_iptu[df_iptu['ANO DO EXERCICIO'] == 2022].groupby('TIPO DE PADRAO DA CONSTRUCAO').agg('count').to_pandas_df()
df_padroes_usos_2022.loc[df_padroes_usos_2022['TIPO DE PADRAO DA CONSTRUCAO'] == 'TERRENO', 'TIPO DE PADRAO DA CONSTRUCAO'] = 'Terreno'
df_padroes_usos_2022['TIPO DE PADRAO DA CONSTRUCAO'] = df_padroes_usos_2022['TIPO DE PADRAO DA CONSTRUCAO'].str.replace('/', ',')
df_padroes_usos_2022

,TIPO DE PADRAO DA CONSTRUCAO,count
0,Residencial vertical - padrão E,56532
1,Indústria - padrão E,124
2,Comercial horizontal - padrão E,208
3,Comercial horizontal - padrão B,91133
4,Comercial vertical - padrão E,19650
5,Comercial vertical - padrão B,54602
6,Residencial horizontal - padrão D,47294
7,Terreno,103197
8,Residencial vertical - padrão C,1079048
9,Comercial horizontal - padrão C,21436


In [ ]:
df_padroes_usos = df_iptu.groupby('TIPO DE PADRAO DA CONSTRUCAO').agg('count').to_pandas_df()
df_padroes_usos.fillna('', inplace=True)
df_padroes_usos['TIPO DE PADRAO DA CONSTRUCAO'] = df_padroes_usos['TIPO DE PADRAO DA CONSTRUCAO'].str.replace('/', ',')
df_padroes_usos

,TIPO DE PADRAO DA CONSTRUCAO,count
0,,26
1,"Templo,Clube,Ginásio ou Estádio esportivo,Muse...",4605
2,Residencial vertical - padrão E,444552
3,Indústria - padrão E,1066
4,Comercial horizontal - padrão E,1669
...,...,...
62,comercial horizontal - Padrão A,276796
63,Residencial vertical - padrão D,2064836
64,Residencial horizontal - padrão C,5175618
65,Comercial vertical - padrão D,302388


In [ ]:
results, ratios = [], []

for i, row in df_padroes_usos.iterrows():

    similaridade = df_padroes_usos_2022['TIPO DE PADRAO DA CONSTRUCAO'].apply(lambda x: SequenceMatcher(None, row['TIPO DE PADRAO DA CONSTRUCAO'], x).ratio())
    
    results.append(df_padroes_usos_2022.iloc[similaridade.idxmax()]['TIPO DE PADRAO DA CONSTRUCAO'])

    ratios.append(similaridade.max())

df_padroes_usos['titulo_padronizado'] = results # Os títulos costumam variar, portanto esse campo será referência para os agrupamentos
df_padroes_usos['relacao_de_assertividade_com_titulo_padronizado'] = ratios

In [ ]:
# df_padroes_usos
df_padroes_usos['tipo_de_uso'] = df_padroes_usos['titulo_padronizado'].str.split("-").apply(lambda x: x[0] if x[0][-1] != ' ' else x[0][:-1])
df_padroes_usos['padrao_de_uso'] = df_padroes_usos['titulo_padronizado'].str.split("-").apply(lambda x: x[-1][-1] if x[-1][-3] == 'o' else '')

df_padroes_usos.loc[(df_padroes_usos.padrao_de_uso == 'A') |
                    (df_padroes_usos.padrao_de_uso == 'B'),
                    'classe_de_uso'] = 'Baixo'

df_padroes_usos.loc[df_padroes_usos.padrao_de_uso == 'C',
                    'classe_de_uso'] = 'Médio'

df_padroes_usos.loc[(df_padroes_usos.padrao_de_uso == 'D') |
                    (df_padroes_usos.padrao_de_uso == 'E') |
                    (df_padroes_usos.padrao_de_uso == 'F'),
                    'classe_de_uso'] = 'Alto'

df_padroes_usos.fillna('', inplace=True)


In [ ]:
df_padroes_usos

,TIPO DE PADRAO DA CONSTRUCAO,count,titulo_padronizado,relacao_de_assertividade_com_titulo_padronizado,tipo_de_uso,padrao_de_uso,classe_de_uso
0,,26,Residencial vertical - padrão E,0.000000,Residencial vertical,E,Alto
1,"Templo,Clube,Ginásio ou Estádio esportivo,Muse...",4605,"Templo,Clube,Ginásio ou Estádio es",0.485714,"Templo,Clube,Ginásio ou Estádio es",,
2,Residencial vertical - padrão E,444552,Residencial vertical - padrão E,1.000000,Residencial vertical,E,Alto
3,Indústria - padrão E,1066,Indústria - padrão E,1.000000,Indústria,E,Alto
4,Comercial horizontal - padrão E,1669,Comercial horizontal - padrão E,1.000000,Comercial horizontal,E,Alto
...,...,...,...,...,...,...,...
62,comercial horizontal - Padrão A,276796,Comercial horizontal - padrão A,0.935484,Comercial horizontal,A,Baixo
63,Residencial vertical - padrão D,2064836,Residencial vertical - padrão D,1.000000,Residencial vertical,D,Alto
64,Residencial horizontal - padrão C,5175618,Residencial horizontal - padrão C,1.000000,Residencial horizontal,C,Médio
65,Comercial vertical - padrão D,302388,Comercial vertical - padrão D,1.000000,Comercial vertical,D,Alto


In [ ]:
outros_tipos = df_padroes_usos.tipo_de_uso.str.contains('Residencial') | df_padroes_usos.tipo_de_uso.str.contains('Comercial') | df_padroes_usos.tipo_de_uso.str.contains('Terreno')

In [ ]:
df_padroes_usos.loc[~outros_tipos, 'tipo_de_uso'] = 'Outros Usos'
df_padroes_usos.loc[~outros_tipos, 'classe_de_uso'] = ''

In [ ]:
df_padroes_usos.rename(columns={'titulo_padronizado':'padrao_construcao'}, inplace=True)

In [ ]:
df_padroes_usos[['tipo_de_uso', 'padrao_de_uso', 'classe_de_uso']]

,tipo_de_uso,padrao_de_uso,classe_de_uso
0,Residencial vertical,E,Alto
1,Outros Usos,,
2,Residencial vertical,E,Alto
3,Outros Usos,E,
4,Comercial horizontal,E,Alto
...,...,...,...
62,Comercial horizontal,A,Baixo
63,Residencial vertical,D,Alto
64,Residencial horizontal,C,Médio
65,Comercial vertical,D,Alto


In [ ]:
df_padroes_usos['tipo_e_classe_de_uso'] = (df_padroes_usos['tipo_de_uso'] + ' ' + df_padroes_usos['classe_de_uso']).str.rstrip()

In [ ]:
# df_iptu = df_iptu[~df_iptu['TIPO DE PADRAO DA CONSTRUCAO'].isna()] 

In [ ]:
df_iptu.fillna(value='Terreno', column_names=['TIPO DE PADRAO DA CONSTRUCAO'], inplace=True)

/home/fernando/dev/dash-iptu/venv/lib/python3.9/site-packages/vaex/arrow/numpy_dispatch.py:72: RuntimeWarning: invalid value encountered in true_divide
  result_data = op['op'](a_data, b_data)


#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,area_ocupada,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"85,961,430",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial
"85,961,431",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial
"85,961,432",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial
"85,961,433",2023,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,0.0,41939500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros


In [ ]:
df_padroes_usos = vaex.from_pandas(df_padroes_usos.loc[df_padroes_usos['TIPO DE PADRAO DA CONSTRUCAO'] != '', ['TIPO DE PADRAO DA CONSTRUCAO',  'tipo_de_uso', 'padrao_de_uso', 'classe_de_uso', 'tipo_e_classe_de_uso']])
df_iptu = df_iptu.join(df_padroes_usos, on='TIPO DE PADRAO DA CONSTRUCAO')

In [ ]:
df_iptu.fillna(value='', column_names=['tipo_de_uso', 'padrao_de_uso', 'classe_de_uso', 'tipo_e_classe_de_uso'], inplace=True)

/home/fernando/dev/dash-iptu/venv/lib/python3.9/site-packages/vaex/arrow/numpy_dispatch.py:72: RuntimeWarning: invalid value encountered in true_divide
  result_data = op['op'](a_data, b_data)


#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,area_ocupada,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"85,961,430",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo
"85,961,431",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo
"85,961,432",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo
"85,961,433",2023,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,0.0,41939500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros,Terreno,,,Terreno


In [ ]:
df_iptu.head(3)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,area_ocupada,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso
0,1995,03812-1,1924,1,13,loja,comercial horizontal - Padrão B,0,135,00-0,135,0,136,001003,001003000100,1,0,0.992647,0,13600,17415,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo
1,1995,03812-1,1944,1,6,loja,comercial horizontal - Padrão A,0,67,00-0,67,0,90,001003,001003000200,1,0,0.744444,0,9000,6298,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo
2,1995,03812-1,1944,1,8,loja,comercial horizontal - Padrão A,0,85,00-0,85,0,105,001003,001003000300,1,0,0.809524,0,10500,7990,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo


In [ ]:
df_iptu[df_iptu['tipo_de_uso'] == 'Comercial vertical']

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,area_ocupada,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso
0,1995,03812-1,1970,4,11.0,loja e residência (predominância comercial),comercial vertical - Padrão B,1.0,802,00-0,802,257,257,001003,001003000700,1.0,257.0,3.1206225680933852,1.0,25700.0,162004.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial vertical,B,Baixo,Comercial vertical Baixo
1,1995,03812-1,1981,2,11.0,loja,comercial vertical - Padrão C,1.0,660,00-0,660,297,303,001003,001003001000,1.0,297.0,2.1782178217821784,0.9801980198019802,30300.0,197340.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial vertical,C,Médio,Comercial vertical Médio
2,1995,03812-1,1924,4,10.0,"'outras edificações do tipo (uso comércio), com ...",comercial vertical - Padrão B,0.0,505,00-0,505,206,285,001003,001003001100,1.0,206.0,1.7719298245614035,0.7228070175438597,28500.0,102010.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial vertical,B,Baixo,Comercial vertical Baixo
3,1995,06156-5,1970,4,12.0,loja e residência (predominância comercial),comercial vertical - Padrão B,1.0,830,00-0,830,270,298,001003,001003001400,1.0,270.0,2.785234899328859,0.9060402684563759,11324.0,167660.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial vertical,B,Baixo,Comercial vertical Baixo
4,1995,06156-5,1983,6,6.0,loja e residência (predominância comercial),comercial vertical - Padrão B,1.0,740,00-0,740,156,162,001003,001003001500,1.0,156.0,4.567901234567901,0.9629629629629629,6156.0,149480.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial vertical,B,Baixo,Comercial vertical Baixo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"4,851,705",2023,06429-7,2016,3,20.0,Escola,Comercial vertical - padrão C,0.95,503,00-0,503,239,508,310096,310096001700,1.0,239.0,0.9901574803149606,0.4704724409448819,939292.0,1332950.0,310,096,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310096275,Serviços,Comercial vertical,C,Médio,Comercial vertical Médio
"4,851,706",2023,05197-7,2002,3,13.3,"'Outras edificações de uso comercial, com utiliz...",Comercial vertical - padrão B,0.81,1161,00-0,1161,394,532,310099,310099000400,1.0,394.0,2.1823308270676693,0.7406015037593985,972496.0,2836323.0,310,099,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310099275,Comercial,Comercial vertical,B,Baixo,Comercial vertical Baixo
"4,851,707",2023,01418-4,1997,3,42.0,Escola,Comercial vertical - padrão B,0.75,2493,00-0,2493,1670,2212,310102,310102008800,1.0,1670.0,1.1270343580470163,0.7549728752260397,3968328.0,6090399.0,310,102,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310102275,Serviços,Comercial vertical,B,Baixo,Comercial vertical Baixo
"4,851,708",2023,02235-7,1974,3,20.0,"'Prédio de escritório ou consultório, não em con...",Comercial vertical - padrão B,0.41,748,00-0,748,348,400,310104,310104002800,1.0,348.0,1.87,0.87,1107200.0,1827364.0,310,104,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310104275,Serviços,Comercial vertical,B,Baixo,Comercial vertical Baixo


In [ ]:
df_iptu.shape

(85961435, 37)

In [ ]:
# df_iptu['tipo_e_classe_de_uso'] = df_iptu['tipo_de_uso'] + ' ' + df_iptu['classe_de_uso']

## Processando as totalizacoes

In [ ]:
gdf_distritos['area'] = gdf_distritos.area
gdf_distritos.to_crs(epsg=4674, inplace=True)

In [ ]:
df_iptu['sq'] = df_iptu.qd_setor + df_iptu.qd_fiscal

In [ ]:
# df_iptu.sq

In [ ]:
# df_iptu.fillna(value='0', column_names=['sq'], inplace=True)
# df_iptu['sq'] = df_iptu['sq'].astype('int')
# df_iptu.categorize('sq', inplace=True)

In [ ]:
# df_iptu.head(1)
df_iptu.fillna(value='0', column_names=['ds_codigo'], inplace=True)
df_iptu['distrito'] = df_iptu['ds_codigo'].astype('int')
df_iptu.categorize('distrito', inplace=True)
# df_iptu.head(1)

/home/fernando/dev/dash-iptu/venv/lib/python3.9/site-packages/vaex/arrow/numpy_dispatch.py:72: RuntimeWarning: invalid value encountered in true_divide
  result_data = op['op'](a_data, b_data)


#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,area_ocupada,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso,distrito
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"85,961,430",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38
"85,961,431",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38
"85,961,432",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38
"85,961,433",2023,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,0.0,41939500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros,Terreno,,,Terreno,38


In [ ]:
# df_iptu['distrito'] = df_iptu['ds_codigo'].astype('int')
df_iptu['ano'] = df_iptu['ANO DO EXERCICIO'].astype('int')
df_iptu.categorize('ano', inplace=True)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,area_ocupada,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso,distrito,ano
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"85,961,430",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023
"85,961,431",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023
"85,961,432",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023
"85,961,433",2023,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,0.0,41939500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros,Terreno,,,Terreno,38,2023


In [ ]:
df_iptu.fillna(value='0', column_names=['od_id'], inplace=True)
df_iptu['od'] = df_iptu['od_id'].astype('int')
df_iptu.categorize('od', inplace=True)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,area_ocupada,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso,distrito,ano,od
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"85,961,430",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275
"85,961,431",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275
"85,961,432",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275
"85,961,433",2023,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,0.0,41939500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros,Terreno,,,Terreno,38,2023,275


In [ ]:
df_iptu.shape

(85961435, 40)

In [ ]:
df_iptu.fillna(value='0', column_names=['ds_cd_sub'], inplace=True)
df_iptu['subprefeitura'] = df_iptu['ds_cd_sub'].astype('int')
df_iptu.categorize('subprefeitura', inplace=True)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,area_ocupada,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso,distrito,ano,od,subprefeitura
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7,9
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7,9
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"85,961,430",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275,15
"85,961,431",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275,15
"85,961,432",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275,15
"85,961,433",2023,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,0.0,41939500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros,Terreno,,,Terreno,38,2023,275,15


In [ ]:
df_iptu.fillna(value='0', column_names=['COD_AED_S'], inplace=True)
df_iptu['censo'] = df_iptu['COD_AED_S'].astype('int')
df_iptu.categorize('censo', inplace=True)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,area_ocupada,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso,distrito,ano,od,subprefeitura,censo
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"85,961,430",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275,15,93
"85,961,431",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275,15,93
"85,961,432",2023,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275,15,93
"85,961,433",2023,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,0.0,41939500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros,Terreno,,,Terreno,38,2023,275,15,93


In [ ]:
# seila = df_iptu['classe_de_uso'] == 'Residencial vertical Baixo'
# df_iptu[seila]
df_iptu.drop(columns=['TIPO DE PADRAO DA CONSTRUCAO'], inplace=True)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,area_ocupada,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso,distrito,ano,od,subprefeitura,censo
0,1995,03812-1,1924,1,13.0,loja,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
1,1995,03812-1,1944,1,6.0,loja,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
2,1995,03812-1,1944,1,8.0,loja,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
3,1995,03812-1,1944,1,6.0,loja,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
4,1995,03812-1,1944,1,7.0,loja,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"85,961,430",2023,02235-7,1978,5,0.0,Apartamento em condomínio,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275,15,93
"85,961,431",2023,02235-7,1978,5,0.0,Apartamento em condomínio,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275,15,93
"85,961,432",2023,02235-7,1978,5,0.0,Apartamento em condomínio,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275,15,93
"85,961,433",2023,19137-0,0,0,0.0,Terreno,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,0.0,41939500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros,Terreno,,,Terreno,38,2023,275,15,93


In [ ]:
padroes_usos = df_padroes_usos['tipo_e_classe_de_uso'].unique()

In [ ]:
# for p in padroes_usos:
#     print(p)

padroes = [
    'Residencial vertical Baixo',
    'Residencial vertical Médio',
    'Residencial vertical Alto',
    'Residencial horizontal Baixo',
    'Residencial horizontal Médio',
    'Residencial horizontal Alto',
    'Comercial vertical Baixo',
    'Comercial vertical Médio',
    'Comercial vertical Alto',
    'Comercial horizontal Baixo',
    'Comercial horizontal Alto',
    'Comercial horizontal Médio',
    'Terreno',
    'Outros Usos',
]

In [ ]:
df = df_iptu

In [ ]:
agg = {
    'Quantidade de Unidades': 'count', 
    'Quantidade de Unidades Condominiais': vaex.agg.count('sqlc',
                                                    selection=df['NUMERO DO CONDOMINIO'] != '00-0'),
    'Tamanho Médio da Unidade Condominial': vaex.agg.mean('AREA CONSTRUIDA', 
                                                    selection=df['NUMERO DO CONDOMINIO'] != '00-0'),
    'Tamanho médio dos Terrenos': vaex.agg.mean('AREA DO TERRENO'),
    'Área Total dos terrenos-lotes': vaex.agg.sum('area_terreno_proporcional'),
    'Área Total Ocupada': vaex.agg.sum('area_ocupada_proporcional'),
    'Área Total Construída':vaex.agg.sum('AREA CONSTRUIDA'),
    'Valor Total dos Terrenos': vaex.agg.sum('valor_terreno'),
    'Valor Total das Construções': vaex.agg.sum('valor_construcao'),
    'CA médio': vaex.agg.mean('ca'),
    'TO médio': vaex.agg.mean('to'),
    'CA médio em lotes condominiais': vaex.agg.mean('ca',
                                                    selection=df['NUMERO DO CONDOMINIO'] != '00-0'),
    'TO médio em lotes condominiais': vaex.agg.mean('to',
                                                    selection=df['NUMERO DO CONDOMINIO'] != '00-0'),
    'CA médio em lotes não condominiais': vaex.agg.mean('ca',
                                                    selection=df['NUMERO DO CONDOMINIO'] == '00-0'),
    'TO médio em lotes não condominiais': vaex.agg.mean('to',
                                                    selection=df['NUMERO DO CONDOMINIO'] == '00-0'),
    'Comprimento Médio da Testada': vaex.agg.mean('TESTADA PARA CALCULO'),
    'Número médio de Pavimentos': vaex.agg.mean('QUANTIDADE DE PAVIMENTOS'),
    'Fator de obsolecência médio': vaex.agg.mean('FATOR DE OBSOLESCENCIA'),
}

In [ ]:
# df_iptu['tipo_e_classe_de_uso'].isna().value_counts()
df_iptu.fillna(value='Outros Usos', column_names=['tipo_e_classe_de_uso'], inplace=True)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,area_ocupada,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso,distrito,ano,od,subprefeitura,censo
0,1995,03812-1,1924,1,13.0,loja,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
1,1995,03812-1,1944,1,6.0,loja,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
2,1995,03812-1,1944,1,8.0,loja,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
3,1995,03812-1,1944,1,6.0,loja,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
4,1995,03812-1,1944,1,7.0,loja,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"85,961,430",2023,02235-7,1978,5,0.0,Apartamento em condomínio,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275,15,93
"85,961,431",2023,02235-7,1978,5,0.0,Apartamento em condomínio,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275,15,93
"85,961,432",2023,02235-7,1978,5,0.0,Apartamento em condomínio,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0033,17.5296,1.4109276345294306,0.2944404412172274,144492.1731,177660.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2023,275,15,93
"85,961,433",2023,19137-0,0,0,0.0,Terreno,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,0.0,41939500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros,Terreno,,,Terreno,38,2023,275,15,93


In [ ]:
df_iptu.head(3)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,area_ocupada,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso,distrito,ano,od,subprefeitura,censo
0,1995,03812-1,1924,1,13,loja,0,135,00-0,135,0,136,001003,001003000100,1,0,0.992647,0,13600,17415,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
1,1995,03812-1,1944,1,6,loja,0,67,00-0,67,0,90,001003,001003000200,1,0,0.744444,0,9000,6298,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
2,1995,03812-1,1944,1,8,loja,0,85,00-0,85,0,105,001003,001003000300,1,0,0.809524,0,10500,7990,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4


In [ ]:
agg_padroes_m2 = {}
agg_padroes_qt = {}

for p in padroes:
    agg_padroes_m2[f"{p} (m2)"] = vaex.agg.sum('AREA CONSTRUIDA', selection=df['tipo_e_classe_de_uso'] == p)
    agg_padroes_qt[f"{p} (qt)"] = vaex.agg.count('AREA CONSTRUIDA', selection=df['tipo_e_classe_de_uso'] == p)

In [ ]:
agg.update(agg_padroes_m2)
agg.update(agg_padroes_qt)

In [ ]:
agg['Terreno (m2)'] = vaex.agg.sum('AREA DO TERRENO', selection=df['tipo_e_classe_de_uso'] == p)
agg['Terreno (qt)'] = vaex.agg.count('AREA DO TERRENO', selection=df['tipo_e_classe_de_uso'] == p)

In [ ]:
list(agg.keys())

['Quantidade de Unidades',
 'Quantidade de Unidades Condominiais',
 'Tamanho Médio da Unidade Condominial',
 'Tamanho médio dos Terrenos',
 'Área Total dos terrenos-lotes',
 'Área Total Ocupada',
 'Área Total Construída',
 'Valor Total dos Terrenos',
 'Valor Total das Construções',
 'CA médio',
 'TO médio',
 'CA médio em lotes condominiais',
 'TO médio em lotes condominiais',
 'CA médio em lotes não condominiais',
 'TO médio em lotes não condominiais',
 'Comprimento Médio da Testada',
 'Número médio de Pavimentos',
 'Fator de obsolecência médio',
 'Residencial vertical Baixo (m2)',
 'Residencial vertical Médio (m2)',
 'Residencial vertical Alto (m2)',
 'Residencial horizontal Baixo (m2)',
 'Residencial horizontal Médio (m2)',
 'Residencial horizontal Alto (m2)',
 'Comercial vertical Baixo (m2)',
 'Comercial vertical Médio (m2)',
 'Comercial vertical Alto (m2)',
 'Comercial horizontal Baixo (m2)',
 'Comercial horizontal Alto (m2)',
 'Comercial horizontal Médio (m2)',
 'Terreno (m2)',
 '

In [ ]:
# df_iptu_grouped_distrito = \
def agrupamento(df, campo):
    return df.groupby([campo, 'ano'],agg=agg)

In [ ]:
# campos = ['distrito', 'subprefeitura', 'od', 'censo', 'sq']
campos = ['distrito', 'subprefeitura', 'od', 'censo', 'sq']

In [ ]:
dfs = []

for campo in campos:
    print(campo)
    df = agrupamento(df_iptu, campo)
    # for u in ['Residencial', 'Comercial', 'Serviços', 'Industrial', 'Outros']:
    #     df[f'Percentual de Uso {u}'] = df[u] / df['Área Total Construída']
    df.export_hdf5(f'data/IPTU-1995-{exercicio_fim}-agrupados-por-{campo}.hdf5')
    dfs.append(df)

distrito
subprefeitura
od
censo
sq


In [ ]:
for i, dist in gdf_distritos.iterrows():
    print(dist.ds_codigo, dist.ds_nome.lower().replace(' ', '-'))
    df_iptu_distrito = df_iptu[df_iptu.distrito == int(dist.ds_codigo)]
    df = agrupamento(df_iptu_distrito, 'sqlc')
    # for u in ['Residencial', 'Comercial', 'Serviços', 'Industrial', 'Outros']:
    #     df[f'Percentual de Uso {u}'] = df[u] / df['Área Total Construída']
    df.export_hdf5(f"data/por_distritos/IPTU-1995-{exercicio_fim}-agrupados-por-sqlc-{dist.ds_codigo}-{dist.ds_nome.lower().replace(' ', '-')}.hdf5")
    # break    

63 pirituba


/home/fernando/dev/dash-iptu/venv/lib/python3.9/site-packages/vaex/arrow/numpy_dispatch.py:72: RuntimeWarning: invalid value encountered in true_divide
  result_data = op['op'](a_data, b_data)


95 sao-domingos
42 jaragua
11 brasilandia
29 freguesia-do-o
21 casa-verde
13 cachoeirinha
50 limao


KeyboardInterrupt: 

In [ ]:
# dfs[0]

In [ ]:
# df_iptu_distrito = vaex.open('data/IPTU-1995-2022-agrupados-por-distrito.hdf5')
# df_iptu_subprefeitura = vaex.open('data/IPTU-1995-2022-agrupados-por-subprefeitura.hdf5')
# df_iptu_od = vaex.open('data/IPTU-1995-2022-agrupados-por-od.hdf5')

# df_iptu_distrito, df_iptu_subprefeitura, df_iptu_od, df_iptu_censo, df_iptu_sq, df_iptu_sqlc = dfs

In [ ]:
# f"{format(df_iptu_distrito['Quantidade de Unidades'].sum(), ',d').replace(',', '.')} seila"

In [ ]:
# df_iptu_sqlc.shape

In [ ]:
# df_iptu_subprefeitura["CA médio"].max()

In [ ]:
# df_iptu_od["CA médio"].max()

In [ ]:
# df_iptu.sqlc